In [2]:
# from matplotlib.pyplot import table
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2
from sqlalchemy import create_engine
import datetime
import dateutil
from google_play_scraper import Sort, reviews
from tqdm.auto import tqdm

In [3]:
class DatabaseConnection:
    def __init__(self):
        connection_info = {
            "host": os.environ.get("PG_HOST", "localhost"),
            "port": os.environ.get("PG_PORT", 5432),
            "db": os.environ.get("PG_DATABASE", "playstore"),
            "user": os.environ.get("PG_USER", "postgres"),
            "password": os.environ.get("PG_PASS", "postgres"),
        }

        # Sql engine
        self.db_engine = create_engine(
            "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
                connection_info["user"],
                connection_info["password"],
                connection_info["host"],
                connection_info["port"],
                connection_info["db"],
            )
        )
        # psql engine
        self.db_conn = psycopg2.connect(
            host=connection_info["host"],
            port=connection_info["port"],
            database=connection_info["db"],
            user=connection_info["user"],
            password=connection_info["password"],
        )

In [8]:
# Get data
def get_data(dttm):
    connection_info = {
        "host": os.environ.get("PG_HOST", "localhost"),
        "port": os.environ.get("PG_PORT", 5432),
        "db": os.environ.get("PG_DATABASE", "playstore"),
        "user": os.environ.get("PG_USER", "postgres"),
        "password": os.environ.get("PG_PASS", "postgres"),
    }

    # Sql engine
    db_engine = create_engine(
        "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
            connection_info["user"],
            connection_info["password"],
            connection_info["host"],
            connection_info["port"],
            connection_info["db"],
        )
    )

    query = f"""
        SELECT "review"."reviewId"
        , "review"."apps"
        , "review"."score"
        , "review"."at"
        , "review"."content"
        , "review"."repliedAt"
        , "sentiment"."clean_text"
        , "sentiment"."sentiment"
        FROM review
        LEFT JOIN sentiment
        ON ("review"."reviewId"="sentiment"."reviewId")
        WHERE "sentiment"."sentiment" is not null
        AND "review"."at" > '{dttm}'
        AND "review"."apps" in ('id.co.cimbniaga.mobile.android','net.myinfosys.PermataMobileX','id.bmri.livin')
        ;
        """
    df = pd.read_sql(query, db_engine)
    df["repliedDurationHrs"] = (df["repliedAt"] - df["at"]).apply(
        lambda x: round(x.seconds / 3600, 2)
    )

    return df

In [12]:
df = get_data('2022-01-01')
df

,reviewId,apps,score,at,content,repliedAt,clean_text,sentiment,repliedDurationHrs
0,8ec5fab4-6c23-4524-ae85-cbb2ba578e3f,id.bmri.livin,5,2022-07-14 10:13:39,ok,2022-08-12 23:54:02,oke,positive,13.67
1,190052db-d4d9-498c-a2e9-06d18c038587,id.bmri.livin,5,2022-07-03 14:46:03,Bgs.,2022-07-04 07:46:45,Bgs,negative,17.01
2,cc36c85f-8e59-42da-bb60-a68cdd7a8cfc,id.bmri.livin,1,2022-09-24 21:08:40,Gagal terus verifikasinya..habis dah pulsa bua...,2022-09-27 16:26:09,Gagal terus verifikasinya habis deh pulsa buat...,negative,19.29
3,9899d237-f581-4f92-978c-af00c9539e07,id.bmri.livin,1,2022-09-24 23:10:02,"Pendaftaran ribet , nga ada menu buat loogin k...",2022-09-27 23:00:34,Pendaftaran ribet enggak ada menu buat loogin...,negative,23.84
4,275618f9-cac9-4750-9373-ca86722cb734,id.bmri.livin,5,2022-09-21 10:02:33,Mantaap,2022-09-21 11:22:01,Mantaap,positive,1.32
...,...,...,...,...,...,...,...,...,...
208147,3740d0a3-29eb-4d1e-b788-6e5403cb4779,id.bmri.livin,1,2022-02-25 22:10:12,Sangat mengecewakan. Awal download lancar2 aja...,2022-03-01 15:52:30,Sangat mengecewakan Awal unduh lancar saja Mak...,negative,17.70
208148,09d16db1-b7c4-41f4-95e6-8615a9e7a5bc,id.bmri.livin,1,2022-02-25 22:10:05,Tidak lebih baik dari living yg biru malah tam...,2022-03-01 15:57:13,Tidak lebih baik dari living yang biru malah t...,negative,17.79
208149,acc0a6e0-df5c-4a17-862d-8c41fa1f2706,id.bmri.livin,1,2022-02-25 21:55:50,Tidak bsa di buka selalu bacaan coba beberapa ...,2022-03-01 19:24:41,Tidak bisa di buka selalu bacaan coba beberapa...,negative,21.48
208150,33eb1b1f-c786-4984-9bd6-2a372f9f2ad3,id.bmri.livin,5,2022-02-25 22:12:50,kok trobel ya,2022-03-01 16:13:13,kok trobel ya,negative,18.01


In [11]:
df.to_csv("payload.csv",index=False)

In [4]:
# Create function get PS Review based on date
app_id = "id.co.bri.brimo"
app_id = "net.myinfosys.PermataMobileX"
conn = DatabaseConnection().db_conn
MAX_COUNT_EACH_FETCH = 200

In [5]:
# helper.status


def get_last_date_db(app_id, conn=None):
    # create query
    date_query = f"""
        SELECT MAX(at)
        FROM review
        WHERE apps = '{app_id}'
        """

    # get dttm from db
    curr = conn.cursor()

    # execute query
    curr.execute(date_query)
    t = curr.fetchall()[0][0]
    if not isinstance(t, datetime.date) and t:
        t = dateutil.parser.parse(t)
    if not t:
        t = dateutil.parser.parse("2023-02-01")
    return t

In [6]:
continuation_token = None
result = []
condition = False
last_date = get_last_date_db(app_id, conn)

In [ ]:
def get_review_by_last_date(app_id, last_date) -> pd.DataFrame:
    results = []
    continuation_token = None
    is_date_match = False

    pbar = tqdm()
    while is_date_match is False:
        try:
            result, continuation_token = reviews(
                app_id,
                count=MAX_COUNT_EACH_FETCH,
                continuation_token=continuation_token,
                lang="id",  # defaults to 'en'
                country="id",  # defaults to 'us'
                sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
                filter_score_with=None,  # defaults to None(means all score)
            )
            results += result
            results_date = results[-1]["at"]
            pbar.update(len(results))
        except:
            continue

        if len(results) % 10000 == 0:
            print(results_date)

        if results[-1]["at"] <= last_date:
            is_date_match = True

    df = pd.DataFrame(result)
    return df

In [ ]:
last_date = dateutil.parser.parse("2023-02-01")

In [ ]:
df = get_review_by_last_date(app_id, last_date)

In [ ]:
df

In [ ]:
results = []
continuation_token = None
is_date_match = False

pbar = tqdm()
while is_date_match is False:
    result, continuation_token = reviews(
        app_id,
        count=MAX_COUNT_EACH_FETCH,
        continuation_token=continuation_token,
        lang="id",  # defaults to 'en'
        country="id",  # defaults to 'us'
        sort=Sort.NEWEST,  # defaults to Sort.MOST_RELEVANT
        filter_score_with=None,  # defaults to None(means all score)
    )
    results += result
    results_date = results[-1]["at"]
    pbar.update(len(results))

    if len(results) % 1000 == 0:
        print(results_date)

    if results[-1]["at"] <= last_date:
        is_date_match = True

In [ ]:
# -1 adalah tanggal yang paling lama , idx 1 adalah tanggal terbaru
results[-1]["at"] <= last_date

# Get data from postgress

In [6]:
dttm = "2022-06-01"
query = f"""
    SELECT "review"."reviewId"
    , "review"."apps"
    , "review"."score"
    , "review"."at"
    , "review"."content"
    , "review"."repliedAt"
    , "sentiment"."clean_text"
    , "sentiment"."sentiment"
    FROM review
    LEFT JOIN sentiment
    ON ("review"."reviewId"="sentiment"."reviewId")
    WHERE "sentiment"."sentiment" is not null
    AND "review"."apps" in ('net.myinfosys.PermataMobileX', 'id.bmri.livin', 'id.co.cimbniaga.mobile.android')
    AND "review"."at" > '{dttm}';
    """

In [7]:
df = pd.read_sql(query, DatabaseConnection().db_conn)


In [8]:
df = df[['apps', 'score', 'content', 'clean_text', 'sentiment']]

In [11]:
df.groupby('apps').size()

apps
id.bmri.livin                     82198
id.co.cimbniaga.mobile.android    13940
net.myinfosys.PermataMobileX       8787
dtype: int64

In [13]:
df.to_csv("3bank.csv", sep=',', index=False)

In [9]:
# Group your data by category
grouped = df.groupby('sentiment')

# Randomly sample 1000 data points from each group
sampled = grouped.apply(lambda x: x.sample(n=3000))

# Concatenate the randomly sampled data from each group into a new dataframe
result = pd.concat([sampled], axis=0)

In [10]:
result.to_csv("raw_data.csv",sep=';')

# Create Bulk Prediction

In [18]:
from transformers import pipeline

In [19]:
pretrained_name = "../models"
_device = 0
model = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name,
    use_auth_token=True,
    device=_device,
)


In [27]:
import pandas as pd

# Sample text data
data = [
    {
        "raw_text": "Hello! This is a sample tweet #sample",
        "cleaned_text": "Hello This is a sample tweet"
    },
    {
        "raw_text": "Hey @user, check out this link: https://example.com",
        "cleaned_text": "Hey check out this link"
    },
    {
        "raw_text": "I'm so happy today 😄",
        "cleaned_text": "I'm so happy today"
    },
    {
        "raw_text": "1234567890",
        "cleaned_text": " "
    },
]

# Create a DataFrame
df = pd.DataFrame(data)

In [28]:
result = model(df['cleaned_text'].to_list())
print(result)

[{'label': 'neutral', 'score': 0.974570631980896}, {'label': 'negative', 'score': 0.7083566784858704}, {'label': 'positive', 'score': 0.9961150884628296}, {'label': 'neutral', 'score': 0.8039116263389587}]


In [29]:
clean_text = ['Mantap tingkatkan', 'BCA mobile kenapa sih sekarang Masa verifikasi sampai  kali sehari Tiap buka aplikasi nya harus verifikasi', 'Sangat baik mudah dan anti ribet', 'mantap', 'Mbanking kebanyakan pemeliharaan sistem terus Mengganggu sangat Ada urusan penting malah begini Ga kayak yang lain', 'Bagus', 'Oke', 'Untuk yang terhormat bapak ibu staf baca dan staf yang bertanggung jawab atas aplikasi baca mobile ini saya ingin sedikit memberi saran alangkah baiknya bila fitur Qris yang di aplikasi baca mobilenya yang bisa menampilkan qr code member baca masing seperti dahulu lagi di samping lebih praktis dan juga lebih cepat Sekian terima kasih', 'Kenapa saya disuruh registrasi ulang ya berpukuh puluh tahun pakai kok tumben saya disuruh ke dilarang baca regis ulang aneh', 'enggak ada hujan dan angin tiba di suruh regust ulng di ATM aduh', 'Apk jelekk Mw buat rekning saja susah sudah beberapa kali daftar tapi transaksi gagal terus Udh habis pulsa banyak Bikin Rugi saja', 'Bagus nih aplikasinya mudah di gunakan dan banyak fiturnya saya kasih  sekali gus', 'W ror terus', 'Update terus', 'Smakin mudah untuk trasaksi', 'Good', 'Mantab tidak rumit', 'Fungsi dan kemudahan transaksi', 'Kok habis dikasih bintang  kemudian di update kok jadi lemot', 'Alhmdllah lebih mudah bertrnsaksi', 'KEBANYAKAN UPDATE', 'Perasaan perbarui Mulu seminggu mau perbarui berapa kali HAH makin malas saya simpan uang di bank ini', 'Setelah diupdate tidak bisa di buka', 'Bagus', 'kebanyaka perbarui mulu perubahan enggak seberapa', 'oke', 'Kok susah sekali unduh Perbaharuan bentar Perbaharuan habis kan Paket aja apa maksudnya ini lemot benar lagi Mau Download Makin parah apa kendala ini  hari Perbaharuan Download enggak bisa', 'Ok bagus', 'Pelayanan baik', 'Apik', 'Sering rusak kadang lemot ketika transaksi padahal mbanking sebelah lancar pernah saya compare jam operasional juga hanya sampai jam  malam enggak bisa diandalkan saat butuh', 'Sangat baik', 'Update mulu heran', 'Baru tanggal  april peningkatan ini sudah peningkatan lagi apa enggak tambah berat inj aplikasi', 'Tdk bisa terbaru padahal jaringannya bagus memuter terus Sdh di restart tapi tetap sama', 'Puas pakai BNI', 'Ok', 'Jos', 'pokoknya mantap', 'mantap', 'Applikasinya selalu rusak', '', 'puas', 'susah', 'mantap', 'Kenapa ya saya transfer di riwayat ada bukti terkirim tapi di bagian transaksi tidak ada Dan manusia yang saya transfer bilang kalau uang nya tidak masuk', 'Buruk sekali sinyal pakai WiFi masih Saja kalau transaksi enggak bisa Dan ada tulisan koneksi terputus Owalah buruk sekali', 'Cuma kasih masukan yah misal seperti mau ganti email atau nomor telepon karena enggak semua bisa ke bank selain waktu hari libur misal bisa ganti lewat telepon CS atau kalau takut keabsahannya bisa lewat vcall jadi bisa tatap muka secara virtual kayaknya itu lebih bagus', 'Kenapa aktivasi gagal terus ketika mau daftar padahal sudah sesuai prosedur', 'ATM nya mudah kalo keluar kota ada di mana sampai di masuk kabupaten', 'Mantap bank negara indonesia', 'mantap memudahkan saya bertransaksi', 'Bermamfaat', 'Memuaskan', 'Cukup memuaskan', 'bagus', 'Mantab', 'oke', 'sangat memuaskan', 'sangat baik mempermudah kami dalam kebutuhan mendadak', 'Its hanya bodoh dan wasting waktu', 'Ganti handphone padahal sama android enggak bisa aktifasi kode otp email nya enggak ada terus tolong lah permudah kami sedikit saja sebagai costumer', 'Ko tidak bisa untuk mendaftarkan sih saya sudah melakukan langkah nya tapi tidak bisa masuk', 'Mantaap', 'Nyaman dipakai', 'perbarui kok terus terusan perbarui itu kalau kayak belum dapat bulan perbarui lagi membuat enggak nyaman', 'Aplikasi yang sangat membantu', 'Keren', 'Saya disuruh verifikasi di ATM sudah bisa tapi malah apakah nya enggak bisa dibuka saja', 'rajin banget meminta di perbarui boros penyimpanan ih', 'Akhir ini aplikasi sering error mulu', 'Gak jelas kalau malam titit', 'Kreen', 'kenapa selalu meminta perbarui dan diupdate pun t bisa ribet amat sih', 'Lebih mudah dan lebih cepat transaksinya', 'Harus terus menerus di Update', 'Baik dan lancar', 'Gimana sih ini sering banget error enggak bisa dibuka dari kemarin malah suruh ngapdate terus tapi tetap enggak bisa dibuka juga aneh', 'ini kenapa saya mau Transfer tapi kok tidak berhasil ke rekening yang lainnya', 'Sangat buruk sekali uda saya perbarui mala meminta perbarui lagi makin buruk makin enggak bisa di andalkan', 'Kenapa tiap kali mau membuat card debit fisik virtual selalu belum berhasil dan tulisannya ulang beberapa saat lagi terus sih dan sudah berulangkali mencoba tetap enggak bisa bisa', 'Bagus', 'Sering perbarui terus', 'Kereen', 'Transaksi lewat e moble mandiri kita sangat terbantu', 'Ini Kepa admin Habis perbarui kok enggak bisa masuk lagi', 'Sangat membantu  Terimakasih', 'Sangat membantu dalam transaksi', 'Ribet sekali datanya terlalu besar sehingga susa sekali untuk aplikasinya pembaruan enggak taunya ke kehapus sehingga semua aktivitas terganggu', 'Update terus adalah repot soalnya loding terus enggak bisa bisa play store memuter Mulu enggak bisa kebuka', 'Senang', 'ini bagaimana ya apknya meminta perbarui terus tapi enggak bisa masuk', 'Sering update tapi malah error enggak bisa dbuka', 'Good', 'Gampang aman dan terpercaya', 'bagus', 'Gua mau login', 'Kurang puas setiap malam pasti gangguan', 'Aplikasi nya bagus tapi sayang kadang suka error mau transfer susahh banget aplikasi tidak bisa dilanjutkan mohon isi tanggal dan waktu otomatis Pls lah ya mohon diperbarui kita yang mau teratas up jadi kesell karena error Gomawo', 'Tiep mau login selalu disuruh perbarui ketika diupdate eh malah sering error andai saja payrollku bukan mandiri wegahh unduh nih aplikasi', 'Mantap', 'Brguna  Mempermudah transaksi', 'Kenapa livin baru san daftar enggak bsadi gunakan muter memuter terus ya', 'Ga bisa transfer', 'Kenapa sekarang enggak ada histori untuk cek emoney bkn cek saldo ya tp cek hystori emoney nya ada dimana skrg aplikasi nya enggak sebagus sebelumnya', 'Kenapa sudah  sunday  ini aplikasi livin enggak bisa dibuka pernah saya uninstall lalu unduh dan bisa digunakan tp  hari kemudian rusak lagi jadi malas pakai aplikasi livin masa harus bolak balik uninstall yang membuat malas memasuki data lagi', 'mantap', 'Top up berhasil dan terkirim pemberitahuan di email sama berhasil terkirim tapi di saldo dana saya top up belum masuk dan saldo masih tetap tidak berubah saya coba cek pesan di dana tetep tidak ada info saldo masuk hmm Uang saya ke mana yang kak Saya kirim email livin jawabnya sudah terkirim Kalau sudah pasti ada pemberitahuan dari dana dan saldo ada', 'Sangat memuaskan', 'aplikasinya bagus banget sangat membantu terima kasih BNI sukses selalu ya', 'sangat membantu sekali', 'Sering error aplikasi tidak dapat dibuka', 'Error nggsk bisa di akses', 'Informatif transaksi mudah', 'Repot mesti perbarui terus untuk memori HP yang full jadi kendala', 'Lelet', 'Update terus hh', 'Mobile banking nya sering eror disuruh perbarui bolak balik lebih lengkap boleh lah Tapi menurut aku lebih enak an yang mobile banking yang lama biru yang kuning ini tambah ribet  Tlong jangan dibuat ribet ya pas lagi butuh transfer tai enggak taunya malah disuruh perbarui lagi sering error pula tlong ditangani ya biar enggak ribet dan lebih baik lagi', 'Mantap', 'Semua berjalan dengan baik', 'Oke banget', 'Kenapa setelah log ini aplikasi ini lalu log out handphone saya selalu lemot padahal ram di handphone saya gb dan menurut saya sangat mumpuni untuk ukuran ram gb terima kasih', 'Cepatan benarkan bugnya enggak bisa transaksi disuruh cek jam otomatis padahal android jamnya selalu otomatis', 'Sering error ketika dibuka clos sendiri', 'Kecewa bulan instal tiap bulan instal seperti aplikasi mainan', 'Aplikasi lemot keseringan perbarui enggak mutu', 'Aplikasinya sering eror kalau di buka ngeloding terus kapan bisa bukanya', 'Gunakan Livin', 'Aplikasinya sangat membantu mempermudah transaksi tapi sering meminta perbarui dan memakan banyak data sekarang saya susah buka karena harus perbarui tapi harus menghapus beberapa aplikasi yang ada', 'Kenapa aplikasi tetap tidak bisa dibuka padahal sudah di perbarui sudah mencoba berkali kali secara berkala tetap tidak bisa peng update an nya pun selalu gagal Mohon segera diatasi karena ini sangat menggangu dan mempersulit pengguna Terimakasih', 'Ok', 'Minta di peningkatan giliran di peningkatan enggak bisa bisa padahal kuota sudah terpakai banyak tetap saja enggak bisa', 'Knp sih aplikasinya perbarui terus Semua memori habis hanya buat perbarui perbarui terus Pas urgent mau pakai tidak bisa digunakan Bikin kesal saja lho Percuma ada apalikasi tidak membantu', 'Membantu banget', 'Luar biasa sangat memudahkan', 'Ini rusak apa bagaimana Kok pin sudah benar malah undefine mulu Sehat', 'bagus pekerjaan continuous improvement', 'mantap', 'sangat bagus', 'bagus sangat membantu', 'Sangat membantu', 'Kenapa login ber kalai kali tidak bisa', 'Aplikasi yang super jelek  setiap mau login aplikasi blank padahal sinyal wifi full sinyal paketan juga full tetap enggak bisa  masa setiap mau login harus uninstall aplikasi dahulu  mengisi data dahulu  baru ada fitur login', 'Bagusan yang biru fitur enggak banyak tapi bisa diandelin Lah ini sih kuning seringan bermasalah mulu Aplikasi kalau ketika jaringan enggak full tiap kali dibuka menutup lagi padhal streaming video lancar saja aneh', 'Livin by Mandiri kagak jelass baru berapa hari penggunaan sudah meminta pajak lah apa lah masa saya ambil pinjam di akulaku sudah cair masa kagak sampai ke rekening saya tiba saja pemberitahuan biaya asuransi penanggungan ke celakaan lah ke matian lah kagak jelass', 'Gogl', 'Ko enggak bisa ada peningkatan yang terbaru nya', 'Kenapa setiap mau teratas up shopee dan pembayaran online lainnya melalui mandiri selalu gagal biasanya selalu berhasil mohon infonya', 'Oklah', 'Sangat mudah dan membatu', 'Setiap malam gangguan terus lama lagi gangguan nya kendala di situ sih padahal sinyal kuat benar kadang pakai WiFi tetap saja enggak bisa', 'Uda enggak bisa transaksi Selalu muncul sedang di proses eh tau nya gagal Makin enggak jelas', 'Transfer SKN Belum Masuk Udh  Hari ini', 'Update terus dengan alasan memperbaiki fitur toh saya saja pakai ini cuma buat lihat saldo Sama transfer saja kebanyakan terima kasih tim yang suka perbarui apakah ini kalian berhasil membuat saya sekeluarga hijrah ke bank sebelah', 'Saldo GK muncul membuat aneh', 'di atas jam  subuh enggak bisa cek saldo dari dahulu enggak ada perkembangan nih apakah', 'Kenapa di saat mau buka apakah nya kadang enggak bisa masuk dengan kata  BELUM BERHASIL TERHUBUNG dan lain lain jika ada permasalah tolong perbaiki', 'Masa harus perbarui melulu', 'Sering rusak', 'Keren Livin by Mandiri Sedikit masukan mohon di tambahkan dalam fitur tarik tunai di kasihimit penarikan mulai  terima kasih', 'Ini kenapa apakah nya enggak jelas gara gara salah pasword login akhirnya memasuki nomor card ATM lagi tapi ini malah di bolak balik suruh membuat pasword sama swafoto  Terus saja begitu sampai gila Ni yang buat apakah enggak mengurus makan gaji buta', 'Aplikasi ketika di buka tidak bisa bisa ketika di pilih muncul terus hilang diulangi beberapa kali baru mau muncul aplikasia padahal sinyal okee bahkan dengan wifi juga seperti itu tolong di perbaiki lagi', 'Sya sgat snang menggunakan livin by mandiri', 'Udah hampir sebulan aku transfer rekening ke bank lain selalu rusak padahal saldo masih cukup Aku tarik juga sekarang kalau saldo sekitar sisa  ribu tidak bisa tarik padahal aku butuh uang anjing Nyesel transfer dari e wallet ke mandiri kalau ternyata seperti begini CEPAT BENAHI NJIING KALAU PERUSAHAANMU TIDAK PENGEN BANGKRUT SAMPAI BESOK AKU TIDAK BISA NARIK MAUPUN TRANSFER SISA SALDO KU INI KUSUMPAHIN BANGKRUT NI PERUSAHAAN HATI AJA SAMA DOA ORANG YANG TERSAKITI YA', 'Gimna sih ya ketika di unistal malah jadi ubah kata sandi pin suruh memasuki lagi tapi enggak masuk perivikasi wajah apalagi terus an salah saja bagaimana ini', 'Bagus dan tidak ribet juga', 'Saldo suka berkurang sendiri padahal bulanan dikenakan kali biaya administrator card debit sebesar ribu tapi sering banget saldo berkurang nominal ribu tanpa ada di riwayat Ga jelas ini aplikasi livin mandiri', 'Aplikasi nya ngpaa sering error tolong boleh cepat diperbaiki', 'Terlalu banyak afdet untuk aplikasi', 'Kok disuru updet terus ya', 'Baguss', 'Bingyng seribg disuruh perbarui tapi paa perbarui disurug install pada perangkst lain', 'Saya senang memakai livin by mandiri Lebih mudah Ngak ribet', 'Ksih bintang  saja lah soalnya aku jengkel setiap mau di buka pasti meminta di perbarui lagi bikin emosi saja lah', 'Ok', 'Kenapa enggak bisa di buka apakah nya brpa kali sudah di peningkatan tapi enggak bisaa memang pant', 'Mudah dan cepat', 'Menambah kemudahan di segala transaksi', 'oke', 'Update mulu perbarui', 'Mending aplikasi yang lama yg sekarang jelek banget mau bayar apa susah banget selalu error mulu kadi mempersulit aplilasinya', 'Eror entah bagaimana padahal sering di pakai', 'Akhir akhir ini sering banget enggak bisa dibuka di saat butuh', 'Apaan nih kok enggak bisa di gunakan transaksi online untuk saat ini', 'Cape meminta pin baru terus setelah di ubah meminta lagi enggak ada puasx', 'aplikasinya minta sering ada perbarui kalo error enggak bisa ada buka lama banget padahal butuh cepat buat transfer', 'Bagus sekali', 'Min ini kenapa layanan transfer bi cepat nya enggak bisa', 'Ini kenapa sih Setelah perbarui malah enggak bisa log ini mental terus terusan', 'baik', 'Sangat membantu banget thanks Mandiri', 'Sip sangat baik', 'Livin by Mandiri ini sering tidak bisa melakukan tranfer atau teratas up di malam hari padahal sering ada keperluan mendesak padalah mesin ATM susah di jangkau saat malam hari mengecewakan', 'Segala jenis transaksi menjadi mudah Thanks Bank Mandiri Semakin maju dalam pelayanan pelanggan', 'Lebih dari cukup', 'Aplikasi cukup mudah di ikuti dan semoga aman', 'kenapa harus pakai perifikasi tambahan ke atm kami yang ada diluar negeri susah untuk melakukannya tolong dpermudahh', 'Cepat dan sangat memudahkan', 'Sangat membantu sayang', 'favi', 'Mudah dan lengkap', 'Mantap', 'Sangat membantu', 'Aiip', 'Memudahkan transaksi', 'Bagus', 'Pagi sudah di perbarui ketika buka aplikasi keluar perbarui jadi mengganggu tolong dong kalo sudah di perbarui jangan lagi ada tulisan perbarui', 'TOP MARKOTOP', 'Malas', 'Buah jambu buah kedondond livin mantap dong', 'Mantaps syekali', 'Maaf saya turunkan langsung bintangnya karena saya membuka aplikasi nya tidak bisa sementara jaringan saya telomsel situasi bagus', 'So helpfull untuk everything', 'Mulai baik', 'Cukup puas', 'Makin perbarui  lengkap layanannya Terimakasih', 'Makin bagus', 'Cukup memuaskan', 'Menu mudah user friendly', 'Memuaskan sekali pelayanan mandiri', 'Sangat bagus', 'Update terus lambat', 'Livin tidak ribet pkknya sangat mudah untuk segala transaksi', 'Ok', 'Ok banget deh', 'Terimakasih livin jadi mempermudah segalanya', 'Transaksi cepat yang terbaik', 'Aplikasi enggak jelas ada saja masalahnya sebelum di transfer bisa login ketika sudah di transfer malah enggak bisa login bagaimana mau mengecek saldo sudah masuk apa belum saya sarankan mending pakai baca saja enggak ribet', 'Ga jelas lama lama mbangkign gangguan Mulu mau isi ulang dana dan lain lain', 'aku kasih bintang  jika pengajuan kita saya disetujuai', 'Transaksi semakin mudah pakai permata bagus dan terbaik', 'aplikasi buruk', 'Aplikasi mengubah data sendiri seperti pasword sudah di ganti beberapa kali di pemberitahuan succes melalui email ataupun melalui aplikasinya sendiri akan tetapi tetap tidak bisa masuk selalu gagal saja', 'Aplikasinya masih kurang bagus jauh dokter pesaingnya pantas nasabah yang lain kasih penilaian yang buruk sy dapat sms penawaran untuk mengubah transaksi card credit ke cicilan tetap melalui aplikasi ini bgtu saya coba ternyata di aplikasi tidak ada transaksi untuk ubah cicilann di telepon ke customer service malah menginformasikan melalui aplkasi udh buang pulsa telepon tapi semua hasilnya nihhiill eiyy kacauu', 'Bug jelek', 'Mudah di pahami joos', 'Saya tidak dapat mengakses kamera caranya bagaimana', 'Featur banyak tidak ada menggunakan', 'Tolong dong di perbaiki setiap kali transfer ke bank lain pakai bi cepat selalu gagal Uang nya enggak masuk Tapi saldo tetap kepotong Besoknya baru uangnya balik lagi tapi sering terjadi seperti ini tolong ya di perbaiki Apalagi uangnya sangat di butuhkan', 'Error mulu nih bos sudah di uninstal lalu unduh lagi masih rusak juga kebekasi naik ojek panas', 'Login gagal terus padahal pasword sudah benar Dikit harus ke kantor mana uang saya sentuh skiming lagi  Ribet ruwet', 'Ga bisa isi saldo gopay tulisannya card tidak ditemukan padahal sudah biasa mengisi dari permata mobile Mohon di pasti', 'Lemot', 'Terima kasih permata mobile kali mudah bertransaksi cinta cinta permata', 'Memudahkan transaksi tanpa harus ke cab', 'Aplikasi nya sangat bagus dan mudah untuk bertransaksi di manapun dan kapanpun', 'Aplikasi praktis dengan tampilan modern Sangat senang bertransaksi dengan aplikasi permata', 'Aplikasi permata mobileX memang Joss', 'Ngajuin kita juta hanya terima juta di potong biaya pencairan ribu lebih tenornya di kasih hanya bulan saja parah ga ada keringanan jatuh tempo tanggal', 'sangat mudah untuk bertransaksi', 'Aplikasi Permata Mobile X  Sangat Membantu Saya Dlm Bertransaksi dengan Mudah Dan Cepat  Skrng Transfer ke semua Bank Lain Gratis biaya Transfer  TERIMA KASIH', 'Pmbox keren-keren sangat membantu untuk transaksi di rumah', 'Gimana konsepnya sih ini aplikasi masa tiap mau verifikasi wajah gagal terus tolonglah di perbaiki saya tidak nyaman kalau kayak begini terus pakai permata Masa tiap mau transfer harus pakai web terus menghabiskan pulsa Sangat tidak efektif kalau begini terus Semoga di baca', 'Transaksi tidak ribet pakai saja Permata mobile kali', 'Bagus memudahkan transaksi kapanpun dan di manapun transfer ke bank lain juga gratis tidak ada maximalnya', 'Permata moblx adalah very very good', 'Woow permata mobil kali semakin keren-keren Transaksi semakin mudah dan gratis biaya transfer tanpa syarat apapun', 'T O P', 'Good', 'kasih bintang  customer service ganggu menelepon ini mulu di suruh ikut program ini itu padahal sudah dikasih tahu enggak mau', 'Aplikasi yang sangat aneh untuk updoad data saja benar enggak bisa astagfirullah', 'Pasword salah terus padahal bner hadeuh', 'Busuk enggak capek banget ngarahin kartu tanda penduduk ke kamera enggak bisa bisa Aplikasi busuk kenapa enggak langsung foto saja kenapa harus sitem yang ngedetect manaan enggak kedetect', 'Lancar', 'Ganti device nama pengguna dan password sudah oke verifikasi wajah gagal terus telepon ke CS permatabank hanya diminta mencoba terus masih terus gagal Telpon lagi jawabannya hanya masalah sistem', 'Kenapa enggk bisa updet padahal kuota saya ada memuter terus seperti gansing', 'Gede banget memakan ruang memori besar Udh cukup bagus Gk perlu perbarui yang handphone nya memori nya kecil kasihan Kn enggak cuma  aplikasi di handphone Makasih Udah begitu sering meminta perbarui mulu LG', 'Cara pengkinian data selain datang ke cabang ada solusi lain enggak admin paimin', 'Gratis biaya transaksi antar bank', 'Ok bagus', 'Bagus', 'Amaziing', 'Baru belajar', 'Susah banget login benar sih demi keamanan tapi sangat merepotkan apa verifikasi otp kartu tanda penduduk belum cukup ya saya sudah lama enggak buka jago makanya bingung sama pertanyaan yang muncul mana saya ingat transaksi terakhir', 'Sangat menakjubkan terima kasih atas pelayanannya yang mengesankan', 'mantap', 'Apa ini makin lama makin tambah rusak', 'mudah sekali', 'Aplikasi paling tidak direkomendasikan cukup sekali transaksi lewat aplikasi ini pengalaman buruk transfer ke bank lain tapi sudah lebih dari sehari saldo belum masuk padahal nominal nya cukup besar bagi yang mau pakai Bank Jago bisa dicek lagi reviewnya ya karena ternyata bukan saya saja yang punya pengalaman seperti ini', 'Aplikasi yang sangat bagus untuk menabung Good joob untuk pelayanan nya', 'layanan baik', 'Oke terbaik', 'sangat membantu dalam semua pembayaran', 'Parah mau bayar cepat gagal terus kasih knyamanan atuh saya mau tranper gagal terus', 'Saya sangat kecewa Udah melakukan persyaratan dan sudah video call tiba tiba melalui email ada pemberitahuan di tolak', 'Bagus tingkatkan tingkatkan jangan buat nasabah kecewa', 'Sy baru bisa jasih binranf dahulu kareba bulan nemahami kualitas apj ini', 'mudah dan cepat', 'Sae banget euy', 'Maaf baru pertama daftar account ketika mau memasuki NIK katanya sudah terdaftar di account bank jago pdhal aku baru pertama ngedaftar', 'Real gratis tanpa biaya untuk transfer ke bank lain', 'Tidak ada penyelesaian Merubah data harus ini itu Sangat sulit', 'awalnya ragu buat buka tabungan di jago ini tapi setelah ada huru hara dari bank sebelah akhirnya mantepin hati buat buka tabungan di jago dan ternyata memang semenyenangkan itu buka tabungan di jago', 'Ok', 'TerBAik', 'mengira bisa pinjm dana', 'Okelah', 'Mantap bank JAGO', 'Loading nya terlalu lama mau ini itu Loading lagi lama pula tolong diperbaiki dan ditingkatkan kecepatannya padahal sinyal internet nya lancar tapi Loading nya lama', 'aplikasi yang mantap dan mudah', 'Apakah tidak ada menu paylater', 'Bagus', 'Alhamdulillah sampai saat ini lancar dan mendukung soal perbankan', 'Bgus', 'The terbaik', 'Good', 'Kok begitu mau masuk malah Force close terus sih padahal lagi perlu uang buat ditarik', 'Masa saya telepon customer service di matiin kartu nya belum sampai sudah dari  hari', 'terbaik', 'lebih mudah transaksi', 'sangat bagus senang sekali menggunakan fiturnya', 'Oke', 'Untuk sementara bagus kasih', 'Semoga pinjam ini lancar dan enggak elegal', 'cepat tanggap', 'Ok', 'Transaksi mudah cepat aman', 'Dimana nih operatornya data saya sudah komplet malah ditolak halo haloo tolong di balas', 'Baguss', 'Recommended secure terjamin mudah dipakai banyak benefit keren-keren', 'RDN ke sesama Bank Jago kurang likuid seharusnya bisa cair sehari juga', 'Sering banget lemot dan error terakhir pocketny enggak kelihatan enggak bisa ngapa in', 'Trimakasi sudah mengembalikan uang ku', 'luar biasa', 'Transaksi sering pending pelayanan customer pelayanan lambat dan kurang memuaskan', 'Ok', 'bagus', 'tfr ke Jago sering PENDING Mutasi ada Tp saldo KOSONG kenapa bisa begitu terus Ada mutasi Ya berarti harus Ada dong saldo nya herann banget sama Bank ini', 'Bagus hanya bunga kantong masih kalah sama kucing', 'Ini jagonya', 'mantap', 'bagus banget apakah nya sangat mudah digunakan dan fitur nya lengkap', 'Banyakan aturan jago tetap keren-keren siabang', 'mantap deh kirim uang langsung sat set pokoknya yang bestlah', 'Good', 'Untuk deposito bagus tidak ada denda kalau ingin menarik lebih cepat pembuatan rekening bank cepat tidak ada biaya administrator bulanan dan juga terdapat event event berhadiah uang tunai', 'terima kasih jago sudah banyak mmbantu  Amanah terus ya jangan kecewakn pelanggan', 'cepat mudah namun sayang sudah bisa QRIS', 'tolng ada petcpt proses nya saya baru saja dowload smpe sudah ada vc udh ada kasih indentitas kartu tanda penduduk masih lama proses nya untuk administrator ini saya Sunardi yang baru daftar dan dowload brusn tadi', 'Sering ada gangguan seperti mengkaitkan account padahal sudah updet', 'Keren', 'Sangat mengecewakan di bank jago saya hanya ingin login kembali ke account bank jago yang dahulu cuma ganti handphone saja sudah enggak bisa login lagi sangat kecewa dengan bank jago', 'Fitur yang diinginkan sudah ada qris', 'BAGUS', 'aplikasi ini ribet', 'puas', 'apknya bangus tapi kok perbarui mulu perasaan lagi mau buat buru eh perbarui lagi fiiuuh', 'apakah enggak jelas masa aku daftar enggak bisa udah menunggu sehari pula daftar nya bagaimana nih apakah yang katanya bintang  padahal sudah ada rencana mabung jadi kayak begini daftar saja enggak bisa', 'bagus membuat mudah transfer dari gopay ke dana tanpa pajak mantap bank jago', 'sejauh ini bagus', 'sederhana dan keren awesome', 'Untuk teratas up e tol selalu gagal', 'sangat membantu', 'sangat membatu unk usaha', 'Lumayan lah', 'Ok deh', 'oke', 'Ok', 'Sampai sejauh ini cukup bagus Semoga ke depannya lebih baik lagi Jago', 'Aplikasi tidak bagus malas pakai aplikasinya', 'oke juga', 'Aplikasi mbanking paling rumit tolol membuat kesal saja dari sekian banyak punya mbanking cuman ini mbanking paling payah rusak di mana kocak', 'Mantap saja deh', 'Knp aplikasinya enggak bisa di pakai ya mau daftar pakai id enggak ada sms yang masuk ikuti cara pakai sidik jadi malah keluar terus', 'Bagus aplikasi nya', 'Sangat membantu', 'Mantaap', 'Fitur Nya lengkap Internet Banking terbagus terlengkap dan terbaik fitur Nya', 'Baik', 'Lebih praktis', 'Kurang membantu  terlalu ribet tidak seperti yang lain', 'Min kok ketika daftar keterangan nomor handphone kamu terblokir bagaimana', 'Ok', 'Bagus lebih mudah segala sesuatu', 'Sulit buat transfer padahal saldo ada aplikasi', 'Good', 'Ribet Bolak balik perbarui', 'Tambah jelek', 'Aplikasinya mudah digunakan dan memiliki banyak fitur yang sangat membantu', 'Sumpah untuk daftarin perangkat susah banget kenapa harus ATM Kalo yang enggak punya ATM bagaimana Suruh membuat lewat cabang terdekat Kalo enggak ada cabang terdekat bagaimana Seharusnya ada opsi lain dong seperti verifikasi muka atau apa begitu Disitu ada saldo saya lo saya lagi butuh enggak bisa diambil Kalo seperti begini sama saja mempersulit nasabah admin Tolong dong solusinya yang lebih bijaksana lagi', 'Gimana nih kok saya login pakai pengguna kenapa enggak jadi setelah itu kodenya enggak terkirim kirim lagi aduh tolong diperbaiki deh', 'Maksutnya apa deh Update malah hancur begini error mulu menghambat urusan manusia tahu enggak', 'Aplikasi kebanyakan rusak nya', 'Kmren lancar kenapa sekarang jadi susah buat transfer mau log ini saja nunggunya lama', 'Update Aplikasi yang sekarang terasa berat dan sering lambat saya tanya teman juga sama lambat tidak seperti Bank yang lain sederhana dan lancar Tolong kembalikan OCTO Mobile yang dahulu terima kasih', 'Aplikasi apaan nih di buka loading mulu lemot benar belajar lagi sana membuat aplikasinya', '', 'selalu force close membuat tambah ribet saja', 'Punya Mobile banking ada salah satu nya Niaga dan Cuma ini saja yang ada buka nya loding terus', 'Sangat membantu', 'Luamyan Untuk UI perlu di ringankan Yg seprlunya saja', 'Sangat mudah dan sangat membantu Selain itu saya juga merasa lebih aman untuk transaksi karena hanya meminta  digit secara acak dari PIN OCTO kita', 'oke', 'Bagus', 'Pertahankan kemudahan dan kenyamanan pengguna', 'mmannttaapp', 'Lebih nyaman octo yang dahulu Kembalikan octo mobile yang dulu', 'Apaansi login sekarang meminta passcode tapi enggak masuk masuk', 'Lebih bagus versi lama', 'Hadeh', 'Di peningkatan kok malah enggak ada rekening card credit terus saya punya  card yang satu CC Octo digital dan yang satunya CC platinum Accor trs kalo pakai CC yang digital enggak bisa soalnya enggak muncul rekening card kreditnya mohon penjelasannya terima kasih', 'Ok', 'Habis perbarui error terus', 'Aplikasi yang sangat bagus mudah di gunakan sekarang tampilan sudah baru Sangat membantu dalam melakukan transaksi trf dan pembayaran Overall sudah bagus dan mohon di pertahankan layanan yang sudah bagus ini', 'Layanan yang buruk', 'Membaik', 'Semoga lebih baik', 'Kenapa enggak bisa login woi aplikasi sampah', 'enggak bisa login sama sekali setelah pembaruan', 'Aplikasi sangat membantu tampilan barunya pun lebih bagus', 'Keren tampilan nya pengguna friendly dan modernisme', 'Bagus sekali', 'Gak jelas menambah susah saja buat login perbarui terus tapi enggak ada gunanya', 'Kenapa enggak bisa registrasi sih', 'Semua sudah normal kembali terbaik', 'Aplikasi apa ini untuk login pun susah jaringan bagus tetap saja pending astaga', 'Terlalu kecewa saat di tanggal janjian verifikasi video call selalu terpental dari aplikasi OCTO Dan sampai sekarang pun sama saja selalu terpental tolong perbaiki', 'Gratis', 'Aplikasi tai Sampahh Gara perbarui aplikasi sekarang aku enggak bisa buka waktu login pengguna ID sudah benar Tp kode verifikasi enggak kekirim ke sms Pdhl pulsa masih banyak Udah seminggu lo seperti begini', 'Transaksi trf cepat', 'Ngotot banget menampilkan rekening ponsel apa menggunakan menampilkan rek ponsel bodoh banget programernya', 'Buat rekpon mau ambil uang saja lama keluar kodenya sudah di mode pesawat tetap delay selang  menit baru terkirim menumpuk seperti sengaja dibuat delay awal buat menghabiskan pulsa sudah enggak kayak dahulu yang bisa di pengaturan lewat internet cepat enggak ada rasa was was menunggu manusia antre belakang intinya sekarang sudah enggak nyaman selamat buat Cimb Niaga atas peningkatan mu manusia jadi pindah rekening goodjob', 'Very bagus', 'habis perbarui malah lemott mau transaksi menunggu lama failed terus bagaimana nih', 'Mungkin kebanyakan fitur aplikasi berat menghambat pekerjaan Balikin saja ke versi lama', 'jadi lemot apakah enggak jelas', 'Cepat', 'Tau lah kenapa banyak menu pada aplikasi sehingga menggunakan juga bingung Sebanyak  aplikasi perbankan ini one really menguras otak dilihat dimaata juga maless', 'Simpel', 'Keren', 'Ok', 'Makin enggak jelas nih aplikasinya', 'Lebih mudah penggunannya', 'Makin membuat kecewa masak meminta di perbarui terus padal sudah perbarui enggak ada seminggu di suruh perbarui lagi', 'Semoga cimb kedepannya nanti bisa makin memperluas kerja sama nya sehingga semua biaya adminnya bisa makin minim hehe', 'yang baru Makin lemot', 'Mau login jadi ribet tadinya adalah tinggal pakai finger print saja', 'Mantap OCTO Mobile', 'Halo pengembang aplikasi Octo Mobile tolong ditingkatkan pelayanan aplikasi ini Saya pengguna baru dari aplikasi ini sebelumny pakai aplikasi dari Bank lain tidak serumit ini dan tidak sering bermasalah Tolong ditingkatkan Kenapa tidak bisa log ini Aplikasi yang seharusnya membantu nasabah dari sebelumnya perbarui juga sering bermasalah Tolong ditingkatkan lagi Saya mau transfer uang jadi tidak bisa ATM CIMB di sini sangat terbatas dan jauh Tolong ditingkatkan', 'Masak setiap Pakai harus perbarui terus belum adaw  sunday  suruh memperbarui terus', 'Top enggak ada duanya', 'Cimb sagat bagus', 'bagus', 'Tampilan baru joss', 'Bener seperti teman teman katakan sudah enggak Isa berkomentar aku Kasih bintang  Ae Mudahan bisa lebih baik kedepannya dan enggak meminta perbarui terus login lagi', 'Mudah an bisa lebih baiik setelah aplikasi nya baru', 'Setelah di perbarui pembaharuan terbaru di tanggal  sesuai tampilan yag baru Saat ini untuk membuka bagian rekeningku membutuhkan waktu yang agak lama Sewaktu login pertama juga agak lama Tidak seperti aplikasi sebelum dilakukan perbarui karena sangat cepat terbukanya Sekiranya mohon agar ada perbaikan Terima kasih', 'Min tolong barusan saya top up dana saldo sudah berkurang tapi kenapa saldo dana nya tidak masuk padahal nomor dana nya sudah benar saya kecewa sih kalo kayak begini', 'Lebih mudah dan cepat', 'tampilan lebih mewah setelah perbarui dan lebih lancar jaringannya', 'Mantap', 'Setelah perbarui aplikasi malah enggak bisa dipakai sudah install uninstall sudah telepon  di blok dan di buka di customer service cabang tetap enggak bisa', 'Aplikasinya bagus mudah di pelajari sangat membantu dan enggak ribet', 'oke', 'Sejak peningkatan sudah berhari hari mBanking enggak bisa tarik ataupun setor tunai Mau login OTP pun harus melalui SMS berbayar Knp nasabah ada membuat ribet begini siy', 'aplikasi versi  bagus bisa digunakan lebih cepat dari yang sebelumnya', 'Setelah di perbarui buruk banget membuat malu ketika bayar aplikasi muterr giliran pembayararan Qriss enggak bisa kembalikan sebelum peningkatan saja lancar enggak ada kendala', 'Bagus', 'Sangat membantu', 'Gajelas banget nih apakah  setelah ada perbarui malah enggak bisa di buka  kocak kamu  kamu di bayar kerja bukan buat membuat manusia susah kocak', 'Ganti perangkat malah enggak bisa masuk', 'Minta perbarui malah enggak bisa dibuka sama sekali sangat mengecewakan Auto uninstall', 'Pembaruan malah menghambat semuanya transaksi yang harus nya bisa dikejar waktu jadi lambat begini', 'Eror terus', 'Setelah pembaruan sering maintance indak kayak dahulu', 'Aplikasinya tambah parah lebih enkan yang dahulu dari pada sekarang masa handphone ram gue  buka aplikasi saja lemotnya meminta ampun gila ini aplikasi Klo kaya begini cara nya aplikasi masih lemot saja mending gue beralih kak bank yang lain deh', 'Ini apknya kenapa suka delay memuter ya  Apkny yang berat apa handphone saya yang jelek', 'setelah di perbarui tambah susah tambah enggak profesional', 'Ok', 'Kenapa setelah perbarui transaksi jadi susah Gagal terus lemot juga Habis perbarui harusya lebih bagus malah jadi buang waktu tahu enggak', 'Aplikasinya mudah di pahami', 'Upgrade masih loading lama ini', 'Aplikasi lemot  menyusahkan susah di andelin menghambat waktu transaksi paki Octo mobile', 'Najis malah enggak bisa log ini Mana mau ambil saldo najis tolol pengin mengomong kasar banget gue', 'Sangat buruk', 'Lemot parah', 'Kacoo naik versi malah makin Lemott Mau login saja Hampir sejam', 'Ini kenapa lemot banget ya untuk loginnya padahal sudah pakai WiFi  lebih mudah yang Octo dahulu diperbarui malah lemot enggak bisa transfer harus login dahulu ribet banget bisa diubah yang dahulu lagi enggak ini', 'Setelah ada perbarui aplikasi malah jadi jelek loading terus enggak kebuka buka', 'Baru di perbarui malah menjadi sangat lama', 'Di perbarui malah lemot bagaimana ini enggak mobile friendly', 'Parah banget ih sudah perbarui malah enggak bisa masuk', 'Saya kasih bintang  dahulu  dikarenakan perbarui jadi transaksi selalu loading nya luama banget banget  Bintang berubah ketika nanti loading nya sudah benar dan tidak lama lagi seperti bulan berikutnya', 'Tolong untuk mobile tunai kalau bisa penerimaan pin otp digraatiskan cukup paket data saja Masa sekelas bank BUMN kalah inovasi sama bank swasta yang sudah gratis pulsa untuk penerimaan pin otp', 'Apk Sangat lemot TOLONG DIPERBAIKI', 'Error', 'Update terbaru sinyal lemot susah buat transaksi', 'di peningkatan versi baru malah enggak becus nih aplikasi enak di versi lama', 'setelah perbarui malah enggak bisa buka aplikasi octo gmn mau transaksi ini', 'Sangat mengecewakan Aplikasi selalu meminta di perbarui tapi setelahnya enggak bisa digunakan error dan saldo enggak cukup eor dan error Hubungi customer pelayanan susah biayanya mahal Harusnya siapkan layanan bebas plz biar pelanggan tidak buang waktu dan uang karena aplikasi yang selalu error', 'Sekarang jadi enggak bagus nih Lama banget loadingnya', 'Aplikasi lemot  Buat transfer memuter terus  Tidak memuaskan sekali', 'Susah banget buat login selalu Eror gagal terus versi baru bapuk', 'Naik versi malah lemot bagusan versi sebelumnya selalu lamban dan terjadi rusak', 'Update malah ribet', 'Pembaruan malah tambah parah lemoot juga', 'Aplikasi sudah terbaru tapi loading setengah mati', 'Aplikasi gagal', 'Aplikasinya jadi lemot banget mau tarik tunai jadi enggak bisa', 'gara gara perbarui mslah enggak bisa login bagaimana sih mana uang semua di situ']

In [39]:
for text in clean_text:
    try:
        model(text)
    except RuntimeError:
        print(text)
        text = " "
        print(model(text))

/home/hvzn/miniconda3/envs/sentiment/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



[{'label': 'neutral', 'score': 0.8039116263389587}]

[{'label': 'neutral', 'score': 0.8039116263389587}]
